In [33]:
from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Bidirectional, Input, TimeDistributed
from keras.layers import LSTM
from keras.datasets import imdb
from keras import optimizers

import pandas as pd
import numpy as np
import os



import torch
from torch.autograd import Variable
from torch import optim,nn


In [34]:
#parameters
maxlen = 30
labels = 2

In [35]:
czech = pd.read_excel('czech.xlsx', encoding='latin',header = None)
czech.columns = ['Name', 'm_or_f']
czech['namelen'] = [len(str(i)) for i in czech['Name']]


ninety = pd.read_csv('name_gender.csv')
ninety = ninety[(ninety['probability']>=0.75)]
ninety.head()



,name,gender,probability
0,Aaban,M,1.0
1,Aabha,F,1.0
2,Aabid,M,1.0
3,Aabriella,F,1.0
4,Aada,F,1.0


## Processing of new ninety file 

In [36]:
ninety['m_or_f'] = list(map(lambda x:x.lower(), ninety['gender']))
ninety['Name'] = list(map(lambda x:x.lower(), ninety['name']))

ninety.drop(['name', 'probability', 'gender'], axis = 1, inplace = True)

In [37]:
n_name = ninety['Name']
nollec = set(' '.join([str(i) for i in n_name]))

In [38]:
cname = czech['Name']
collect = []


for i in range(len(cname)):
    collect.extend(list(str(cname[i]).lower()))
# collect.extend(['END'])
collect = set(collect)

czech.head()

,Name,m_or_f,namelen
0,Abigail,f,7
1,Ada,f,3
2,Adalberta,f,9
3,Adéla,f,5
4,Adelaida,f,8


In [39]:
data_set = pd.read_csv("gender_data.csv",header=None)
data_set.columns = ['name','m_or_f']
data_set['namelen']= [len(str(i)) for i in data_set['name']]
data_set1 = data_set[(data_set['namelen'] >= 2) ]

In [40]:
ninety.groupby('m_or_f')['Name'].count()

m_or_f
f    59103
m    33465
Name: Name, dtype: int64

In [41]:
names = data_set['name']
gender = data_set['m_or_f']
vocab = set(' '.join([str(i) for i in names]))
vocab.add('END')
vocab = vocab.union(collect)
vocab = vocab.union(nollec)
len_vocab = len(vocab)

In [42]:
print(vocab)
print("vocab length is ",len_vocab)
print ("length of data_set is ",len(data_set1))

{'č', 'o', 'u', 'á', 'ě', 'r', 'f', 'ň', 'b', 'h', 'ř', 'é', 'j', '.', '4', 'i', 'p', 'a', 'q', 'ž', '2', 'c', '9', 'l', 'v', 'g', '3', 'ť', 'k', 'w', 'x', 'y', '8', 'ý', ' ', 's', 'ů', 'm', 'í', 'z', 'e', '7', '0', 't', '5', 'd', 'n', 'ó', 'ď', 'š', '6', 'ú', '1', 'END'}
vocab length is  54
length of data_set is  15226


In [43]:
char_index = dict((c, i) for i, c in enumerate(vocab))

In [44]:
print(char_index)

{'č': 0, 'o': 1, 'u': 2, 'á': 3, 'ě': 4, 'r': 5, 'f': 6, 'ň': 7, 'b': 8, 'h': 9, 'ř': 10, 'é': 11, 'j': 12, '.': 13, '4': 14, 'i': 15, 'p': 16, 'a': 17, 'q': 18, '2': 20, 'c': 21, '9': 22, 'l': 23, '0': 42, 'v': 24, 'g': 25, '3': 26, 'ť': 27, 'k': 28, 'w': 29, 'e': 40, 'y': 31, 'ý': 33, ' ': 34, 's': 35, 'ů': 36, 'm': 37, 'í': 38, 'z': 39, '7': 41, 'END': 53, 't': 43, 'ž': 19, 'x': 30, 'd': 45, 'n': 46, 'ó': 47, 'š': 49, '6': 50, 'ú': 51, '1': 52, 'ď': 48, '5': 44, '8': 32}


In [45]:
#train test split
msk = np.random.rand(len(data_set1)) < 0.9
train = data_set1[msk]
test = data_set1[~msk]     

msk = np.random.rand(len(czech)) < 0.9


vtrain = czech[msk]
vtest = czech[~msk]

msk = np.random.rand(len(ninety)) < 0.9

        
ntrain = ninety[msk]
ntest = ninety[~msk]

In [46]:
def set_flag(i):
    tmp = np.zeros(len_vocab);
    tmp[i] = 1
    return(tmp)

In [47]:
set_flag(3)

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

#### modify the code above to also convert each index to one-hot encoded representation

# These are set of indian names, just for adding a little robustness and to construct that latent structure in names 

In [48]:
#take data_set upto max and truncate rest
#encode to vector space(one hot encoding)
#padd 'END' to shorter sequences
#also convert each index to one-hot encoding
train_x = []
train_y = []
trunc_train_name = [str(i)[0:maxlen] for i in train.name]
for i in trunc_train_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    train_x.append(tmp)
for i in train.m_or_f:
    if i == 'm':
        train_y.append([1,0])
    else:
        train_y.append([0,1])
    
train_x=np.asarray(train_x)
train_y=np.asarray(train_y)

In [49]:
test_x = []
test_y = []
trunc_test_name = [str(i)[0:maxlen] for i in test.name]
for i in trunc_test_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    test_x.append(tmp)
for i in test.m_or_f:
    if i == 'm':
        test_y.append([1,0])
    else:
        test_y.append([0,1])
    
test_x = np.asarray(test_x)
test_y = np.asarray(test_y)

# These are Czech names, well, that's what the program is all about 

In [50]:
vtrain_x = []
vtrain_y = []

train_name = [str(i) for i in vtrain.Name]
for i in train_name:
    tmp = [set_flag(char_index[j]) for j in str(i.lower())]
    for k in range(0, maxlen - len(str(i))):
        tmp.append(set_flag(char_index['END']))
    vtrain_x.append(tmp)
for i in vtrain.m_or_f:
    if i == 'm':
        vtrain_y.append([1,0])
    else:
        vtrain_y.append([0,1])
vtrain_x = np.asarray(vtrain_x)
vtrain_y = np.asarray(vtrain_y)

In [51]:
vtest_x = []
vtest_y = []

train_name = [str(i) for i in vtest.Name]
for i in train_name:
    tmp = [set_flag(char_index[j]) for j in str(i.lower())]
    for k in range(0, maxlen - len(str(i))):
        tmp.append(set_flag(char_index['END']))
    vtest_x.append(tmp)
for i in vtest.m_or_f:
    if i == 'm':
        vtest_y.append([1,0])
    else:
        vtest_y.append([0,1])
vtest_x = np.asarray(vtest_x)
vtest_y = np.asarray(vtest_y)

# This is just collection of names from all part of world, majority of my dataset 

In [52]:
ntrain_x = []
ntrain_y = []

train_name = [str(i) for i in ntrain.Name]
for i in train_name:
    tmp = [set_flag(char_index[j]) for j in str(i.lower())]
    for k in range(0, maxlen - len(str(i))):
        tmp.append(set_flag(char_index['END']))
    ntrain_x.append(tmp)
for i in ntrain.m_or_f:
    if i == 'm':
        ntrain_y.append([1,0])
    else:
        ntrain_y.append([0,1])
ntrain_x = np.asarray(ntrain_x)
ntrain_y = np.asarray(ntrain_y)

In [53]:
ntest_x = []
ntest_y = []

train_name = [str(i) for i in ntest.Name]
for i in train_name:
    tmp = [set_flag(char_index[j]) for j in str(i.lower())]
    for k in range(0, maxlen - len(str(i))):
        tmp.append(set_flag(char_index['END']))
    ntest_x.append(tmp)
for i in ntest.m_or_f:
    if i == 'm':
        ntest_y.append([1,0])
    else:
        ntest_y.append([0,1])
ntest_x = np.asarray(ntest_x)
ntest_y = np.asarray(ntest_y)

# Combining all training set, and all test set 

In [54]:
print("These are all training data!")
print("Shape of indian training name:", train_x.shape)
print("Shape of Czech training name:", vtrain_x.shape)
print('Shape of all world training name:',ntrain_x.shape)

These are all training data!
Shape of indian training name: (13776, 30, 54)
Shape of Czech training name: (1132, 30, 54)
Shape of all world training name: (83196, 30, 54)


In [55]:
print("Combining them all...")
net_train_x = np.concatenate([train_x, vtrain_x, ntrain_x], axis = 0)
net_train_y = np.concatenate([train_y, vtrain_y, ntrain_y], axis = 0)
print("Now net shape is:", net_train_x.shape)
print("Now they're combined, but we must shuffle them too...shuffling")
np.random.shuffle(net_train_x)
np.random.shuffle(net_train_y)
print("Shuffled!!")

Combining them all...
Now net shape is: (98104, 30, 54)
Now they're combined, but we must shuffle them too...shuffling
Shuffled!!


In [56]:
print("These are all test data!")
print("Shape of indian test name:", test_x.shape)
print("Shape of Czech test name:", vtest_x.shape)
print('Shape of all world test name:',ntest_x.shape)

These are all test data!
Shape of indian test name: (1450, 30, 54)
Shape of Czech test name: (118, 30, 54)
Shape of all world test name: (9372, 30, 54)


In [57]:
print("Combining them all...")
net_test_x = np.concatenate([test_x, vtest_x, ntest_x], axis = 0)
net_test_y = np.concatenate([test_y, vtest_y, ntest_y], axis = 0)
print("Now net shape is:", net_test_x.shape)
print("Now they're combined, but we must shuffle them too...shuffling")
np.random.shuffle(net_test_x)
np.random.shuffle(net_test_y)
print("Shuffled!!")

Combining them all...
Now net shape is: (10940, 30, 54)
Now they're combined, but we must shuffle them too...shuffling
Shuffled!!


## Model in PyTorch 

In [68]:
class gender(nn.Module):
    def __init__(self, input_dim = 54, hidden_dim = 524, output_dim = 2, n_layers = 2, lr = 0.001):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first = True, bidirectional = True)
        self.linear = nn.Linear(hidden_dim * 2, output_dim)
        
    def forward(self, x, hidden):
        lstmout, hidden = self.lstm.forward(x, hidden)
        output = self.linear(lstmout)
        return output, hidden
    
    def init_hidden(self, batch_size):
        
        weight = next(self.parameters()).data
        
        hidden = (weight.new(self.n_layers * 2, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers * 2, batch_size, self.hidden_dim).zero_())
        return hidden

In [69]:
mynet = gender(input_dim = 54, output_dim = 2, hidden_dim = 50, n_layers = 2)

In [70]:
def train(net, trainx, trainy, testx, testy, batch_size = 500, epochs = 50, lr = 0.01, clip = 0.01, print_every = 10):
    
    net.train()
    net.double()
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr = lr)
    
    net
    counter = 0
    val_losses = []
    train_permutation = torch.randperm(trainx.shape[0])
    test_permutation = torch.randperm(testx.shape[0])

    
    for i in range(epochs):
        counter += 1
        
        #Initializing hidden state
        hd = net.init_hidden(batch_size)
        for i in range(0,trainx.shape[0], batch_size):
            
            indices = train_permutation[i:i+batch_size]

        
            inputs, targets = torch.from_numpy(trainx[indices]), torch.from_numpy(trainy[indices])
            hd = tuple([each.data for each in hd])

            net.zero_grad()

            outputs, hd = net(inputs, hd)

            loss = criterion(outputs, targets)
            loss.backward()

            optimizer.step()

            if counter % print_every == 0:
                net.eval()
                val_h = net.init_hidden(batch_size)
                
                for j in range(0, testx.shape[0], batch_size):
                    
                    indic = test_permutation[j:j + batch_size]
                    vinputs, vtargets = torch.from_numpy(testx[indic]), torch.from_numpy(testy[indic])

                    val_h = tuple([each.data for each in val_h])

                    output_h, val_h = net(vinputs, val_h)
                    val_loss = criterion(output_h, vtargets)
                    val_losses.append(val_loss.item())

            net.train()

            print("Epoch: {}/{} ...".format(i+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.4f}...".format(loss.item()),
                  "Val Loss:{:.4f}".format(np.mean(val_losses)))

In [71]:
hidden = train(mynet, trainx = net_train_x, trainy = net_train_y, testx = net_test_x, testy = net_test_y, )

Epoch: 1/50 ... Step: 1... Loss: 3.3926... Val Loss:nan
Epoch: 501/50 ... Step: 1... Loss: 3.2703... Val Loss:nan
Epoch: 1001/50 ... Step: 1... Loss: 2.9751... Val Loss:nan
Epoch: 1501/50 ... Step: 1... Loss: 2.7687... Val Loss:nan
Epoch: 2001/50 ... Step: 1... Loss: 2.6628... Val Loss:nan
Epoch: 2501/50 ... Step: 1... Loss: 2.6826... Val Loss:nan
Epoch: 3001/50 ... Step: 1... Loss: 2.3607... Val Loss:nan
Epoch: 3501/50 ... Step: 1... Loss: 2.0570... Val Loss:nan
Epoch: 4001/50 ... Step: 1... Loss: 1.9140... Val Loss:nan
Epoch: 4501/50 ... Step: 1... Loss: 2.4118... Val Loss:nan
Epoch: 5001/50 ... Step: 1... Loss: 1.6517... Val Loss:nan
Epoch: 5501/50 ... Step: 1... Loss: 1.6997... Val Loss:nan
Epoch: 6001/50 ... Step: 1... Loss: 1.7232... Val Loss:nan
Epoch: 6501/50 ... Step: 1... Loss: 1.4841... Val Loss:nan
Epoch: 7001/50 ... Step: 1... Loss: 1.6448... Val Loss:nan
Epoch: 7501/50 ... Step: 1... Loss: 1.8735... Val Loss:nan
Epoch: 8001/50 ... Step: 1... Loss: 1.5239... Val Loss:nan
E

RuntimeError: Expected hidden[0] size (4, 104, 50), got (4, 500, 50)

In [ ]:
a = 1

#### build model in keras ( a stacked LSTM model with many-to-one arch ) here 30 sequence and 2 output each for one category(m/f)

In [ ]:
# #build the model: 2 stacked LSTM
# print('Build model...')
# input_bilstm=Input(shape = (maxlen,len_vocab))
# bi_one = Bidirectional(LSTM(1024, return_sequences=True))(input_bilstm)
# drop1 = Dropout(0.2)(bi_one)
# bi_two = Bidirectional(LSTM(1024, return_sequences=False))(drop1)
# drop2 = Dropout(0.2)(bi_two)
# output = Dense(2, activation='softmax')(drop2)
# model = Model(input_bilstm, output)


# opt = optimizers.adam(lr = 0.01)
# model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])


#build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen,len_vocab)))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(2))
model.add(Activation('softmax'))
opt = optimizers.SGD(lr=0.1)

model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
print("Model Built")

In [ ]:
batch_size=500
model.fit(net_train_x, net_train_y,
          batch_size=batch_size,
          epochs=50,
          validation_data=(net_test_x, net_test_y)
         )
model.save('Martin_program.h5')

In [ ]:
score, acc = model.evaluate(vtest_x, vtest_y)
print('Test score:', score)
print('Test accuracy:', acc)

In [29]:
name=["riya"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i.lower())]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[0.30979005, 0.6902099 ]], dtype=float32)

In [30]:
pred

array([[0.30979005, 0.6902099 ]], dtype=float32)

#### Lets train more, clearly some very simple female names it doesnt get right like mentioned above (inspite it exists in training data)

In [31]:
score, acc = model.evaluate(test_X, test_Y)
print('Test score:', score)
print('Test accuracy:', acc)

NameError: name 'test_X' is not defined

In [32]:
name=["sandhya","jaspreet","rajesh","kaveri","aditi deepak","arihant","sasikala","aditi","ragini rajaram"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[0.30979005, 0.69021   ],
       [0.30979005, 0.69021   ],
       [0.30979005, 0.69021   ],
       [0.30979005, 0.69021   ],
       [0.30979005, 0.69021   ],
       [0.30979005, 0.69021   ],
       [0.30979005, 0.69021   ],
       [0.30979005, 0.69021   ],
       [0.30979005, 0.69021   ]], dtype=float32)

In [33]:
name=["abhi","abhi deepak","mr. abhi"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[0.4178515 , 0.5821485 ],
       [0.37405205, 0.62594795],
       [0.02188767, 0.9781123 ]], dtype=float32)

In [34]:
name=["rajini","rajinikanth","mr. rajini"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[0.09026915, 0.90973085],
       [0.9785337 , 0.02146639],
       [0.03325909, 0.9667409 ]], dtype=float32)

In [35]:
#save our model and data
model.save_weights('gender_model',overwrite=True)
train.to_csv("train_split.csv")
test.to_csv("test_split.csv")

In [36]:
evals = model.predict(test_X)
prob_m = [i[0] for i in evals]

In [37]:
out = pd.DataFrame(prob_m)
out['name'] = test.name.reset_index()['name']
out['m_or_f']=test.m_or_f.reset_index()['m_or_f']

In [38]:
out.head(10)
out.columns = ['prob_m','name','actual']
out.head(10)
out.to_csv("gender_pred_out.csv")